# Number of Features and Overfitting

In [2]:
# code from feature_selection/find_signature.py
import pickle
import numpy
import os
numpy.random.seed(42)

In [5]:
os.chdir('C:/Users/rsh15/Desktop/seunghuni/udacity/ud120-projects/')
words_file = "text_learning/your_word_data.pkl" 
authors_file = "text_learning/your_email_authors.pkl"
word_data = pickle.load( open(words_file, "r"))
authors = pickle.load( open(authors_file, "r") )

In [8]:
from sklearn.model_selection import train_test_split
features_train, features_test, labels_train, labels_test = train_test_split(word_data, authors, test_size=0.1,
                                                                            random_state=42)

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                             stop_words='english')
features_train = vectorizer.fit_transform(features_train)
features_test  = vectorizer.transform(features_test).toarray()

In [9]:
features_train = features_train[:150].toarray()
labels_train   = labels_train[:150]

# Accuracy of your Decision Tree

In [17]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(features_train,labels_train).score(features_test,labels_test)

0.9590443686006825

# Identify the Most Powerful Features

In [20]:
import numpy as np
import matplotlib.pyplot as plt
np.sort(dt.feature_importances_)

array([0.        , 0.        , 0.        , ..., 0.07495003, 0.13402829,
       0.76470588])

In [34]:
dfs = [np.arange(0,len(dt.feature_importances_)),dt.feature_importances_]

In [39]:
pd.DataFrame(dfs).T.sort_values(1,ascending=False).head(5)

,0,1
33614,33614.0,0.764706
33953,33953.0,0.134028
26874,26874.0,0.074950
19652,19652.0,0.026316
25244,25244.0,0.000000


# Use TfIdf to get the Most important word

In [73]:
vectorizer.get_feature_names()[33614]

u'sshacklensf'

# Remove, Repeat

In [97]:
# preperation
# code from text learning - https://github.com/seunghunii/simple_examples/blob/master/2.Lesson10-Text%20Learning.ipynb
from nltk.stem.snowball import SnowballStemmer
import string
stemmer = SnowballStemmer('english')
os.chdir('C:/Users/rsh15/Desktop/seunghuni/udacity/ud120-projects/text_learning/')

In [98]:
def parseOutText(f):
    """ given an opened email file f, parse out all text below the
        metadata block at the top
        (in Part 2, you will also add stemming capabilities)
        and return a string that contains all the words
        in the email (space-separated) 
        
        example use case:
        f = open("email_file_name.txt", "r")
        text = parseOutText(f)"""
    
    f.seek(0)  ### go back to beginning of file (annoying)
    all_text = f.read()

    ### split off metadata
    content = all_text.split("X-FileName:")
    words = ""
    if len(content) > 1:
        ### remove punctuation
        text_string = content[1].translate(string.maketrans("", ""), string.punctuation)

        ### project part 2: comment out the line below
        
        ### split the text string into individual words, stem each word,
        ### and append the stemmed word to words (make sure there's a single
        ### space between each stemmed word)
        for word in text_string.split():
            # stem the word and add it to words
            words += stemmer.stem(word) + ' '       
        
    return words[:-1]

In [111]:
def get_rid_of_signature_words(sw):
    with open("from_sara.txt", "r") as from_sara, open("from_chris.txt", "r") as from_chris:

        from_data = []
        word_data = []

        ### temp_counter is a way to speed up the development--there are
        ### thousands of emails from Sara and Chris, so running over all of them
        ### can take a long time
        ### temp_counter helps you only look at the first 200 emails in the list so you
        ### can iterate your modifications quicker
        temp_counter = 0


        for name, from_person in [("sara", from_sara), ("chris", from_chris)]:
            for path in from_person:
                ### only look at first 200 emails when developing
                ### once everything is working, remove this line to run over full dataset

                #temp_counter += 1
                if temp_counter < 200:
                    path = os.path.join('..', path[:-1])

                    with open(path, 'r') as email:
                        ### use parseOutText to extract the text from the opened email
                        text = parseOutText(email)

                        ### use str.replace() to remove any instances of the words
                        ### ["sara", "shackleton ", "chris", "germani"]
                        for word in sw:
                            if(word in text):
                                text = text.replace(word, "")

                        ### append the text to word_data
                        tmp_txt = text.replace('\n',' ').strip()
                        #tmp_txt = tmp_txt.replace(' {2,}','').lower()
                        word_data.append(tmp_txt)

                        ### append a 0 to from_data if email is from Sara, and 1 if email is from Chris
                        if name=='sara':
                            from_data.append(0)
                        else:
                            from_data.append(1)
    return word_data, from_data
                            
sw = ["sara", "shackleton", "chris", "germani",'sshacklensf']
word_data, from_data = get_rid_of_signature_words(sw)

In [112]:
from sklearn.model_selection import train_test_split
new_features_train, new_features_test, new_labels_train, new_labels_test = train_test_split(word_data, from_data,
                                                                                            test_size=0.1,random_state=42)

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                             stop_words='english')
new_features_train = vectorizer.fit_transform(new_features_train)
new_features_test  = vectorizer.transform(new_features_test).toarray()

In [113]:
new_features_train = new_features_train[:150].toarray()
new_labels_train   = new_labels_train[:150]

In [114]:
dt = DecisionTreeClassifier()
dt.fit(new_features_train,new_labels_train).score(new_features_test,new_labels_test)

0.9687144482366326

In [115]:
dfs = [np.arange(0,len(dt.feature_importances_)),dt.feature_importances_]
pd.DataFrame(dfs).T.sort_values(1,ascending=False).head(10)

,0,1
14343,14343.0,0.666667
8674,8674.0,0.162602
16268,16268.0,0.093809
14337,14337.0,0.050607
13862,13862.0,0.026316
25250,25250.0,0.000000
25249,25249.0,0.000000
25248,25248.0,0.000000
25247,25247.0,0.000000
25246,25246.0,0.000000


In [116]:
vectorizer.get_feature_names()[14343]

u'cgermannsf'

# Checking Important Feature Again

In [117]:
sw = ["sara", "shackleton", "chris", "germani",'sshacklensf','cgermannsf']
word_data, from_data = get_rid_of_signature_words(sw)

In [118]:
from sklearn.model_selection import train_test_split
new_features_train2, new_features_test2, new_labels_train2, new_labels_test2 = train_test_split(word_data, from_data,
                                                                                            test_size=0.1,random_state=42)

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                             stop_words='english')
new_features_train2 = vectorizer.fit_transform(new_features_train2)
new_features_test2  = vectorizer.transform(new_features_test2).toarray()

In [119]:
new_features_train2 = new_features_train2[:150].toarray()
new_labels_train2   = new_labels_train2[:150]

In [120]:
dt = DecisionTreeClassifier()
dt.fit(new_features_train2,new_labels_train2).score(new_features_test2,new_labels_test2)

0.8253697383390216

In [121]:
dfs = [np.arange(0,len(dt.feature_importances_)),dt.feature_importances_]
pd.DataFrame(dfs).T.sort_values(1,ascending=False).head(10)

,0,1
21323,21323.0,0.363636
18849,18849.0,0.186927
11975,11975.0,0.105379
22546,22546.0,0.084069
29690,29690.0,0.061295
33905,33905.0,0.047407
17248,17248.0,0.042667
14480,14480.0,0.026280
30831,30831.0,0.025529
26873,26873.0,0.024810


In [122]:
vectorizer.get_feature_names()[14343]

u'cgermanyenroncom'